# nltk NaiveBayes + scikit tfidf mit NaiveBayes

In [14]:
def word_feats(words):
    return dict([(word, True) for word in words.split()])

def prep_text(String):
    return re.sub('[\s]+',' ',String).replace('.','').strip().split(' ')

def classify(data,split_ratio,SEED,db):
    
    import sqlite3
    import re
    import pandas as pd
    from nltk.classify import NaiveBayesClassifier
    import random

    amazon_sql = 'data/amazon/amazon.db' #db -> id+rating

    # Lade Amazon_raw
    conn=sqlite3.connect(amazon_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID, text,rating FROM dvd')
    getrating={ID:rating for ID, text,rating in cur}
    conn.close() 

    count_tok=0
    count_utok=set()

    delete=[]
    for key,value in data.items():
        try:
            text=re.sub('[\s]+',' ',data[key].replace('.','')).strip().lower()
            temp=text.split(' ')
            count_utok.update(set(temp))
            count_tok+=len(temp)

            data[key]=text
        except:
            delete.append(key)
            
    for key in delete:
        data.pop(key, None)

    count_utok=len(count_utok)
    count_tok,count_utok
    
    posrevs=len(['' for ID in data.keys() if int(getrating[ID][0])>3])
    negrevs=len(['' for ID in data.keys()  if int(getrating[ID][0])<3])
    

    IDs=list(data.keys())
    if type(SEED) is int:
        random.seed(SEED)
    random.shuffle(IDs)

    l=int(len(IDs)*split_ratio)
    trainIDs=IDs[:l]
    testIDs=IDs[l:]
    lid=len(IDs)

    posids=[data[ID] for ID in trainIDs if int(getrating[ID][0])>3]
    negids=[data[ID] for ID in trainIDs if int(getrating[ID][0])<3]
    
    lpr=len(posids)
    lnr=len(negids)
    
    
    ufeats=set()
    for text in posids+negids:
        try:
            text=re.sub('[\s]+',' ',text.replace('.','')).strip().lower()
            temp=text.split(' ')
            ufeats.update(set(temp))
        except:
            pass
    ufeats=len(ufeats)
    
    if db!='tfidf':
        pos_feats = [(word_feats(f), 'positive') for f in posids ]
        neg_feats = [(word_feats(f), 'negative') for f in negids ]
        trainfeats = pos_feats + neg_feats

        classifier = NaiveBayesClassifier.train(trainfeats)

        dict_test={'pos|pos':0,'pos|neg':0,'neg|pos':0,'neg|neg':0,}


        for ID in testIDs:
            rating=int(getrating[ID][0])
            if rating<3:
                rating='neg'
            elif rating>3:
                rating='pos'
            true_rating=rating

            rating=[(label, classifier.prob_classify(word_feats(data[ID])).prob(label)) for label in ['positive','negative']][0][1]
            if rating<=0.5:
                rating='neg'
            elif rating>=0.5:
                rating='pos'
            pred_rating=rating
            dict_test[true_rating+'|'+pred_rating]+=1

            test_matrix = pd.DataFrame(index=['pos','neg'], columns=['pos','neg'])
            test_matrix = test_matrix.fillna('')

            for tag,n in dict_test.items():
                col, row = tag.split('|')
                test_matrix.at[col, row] = n

        accuracy=float(dict_test['pos|pos']+dict_test['neg|neg'])/(sum(dict_test.values()))
        recall=float(dict_test['pos|pos'])/(dict_test['pos|pos']+dict_test['pos|neg'])
        precision=float(dict_test['pos|pos'])/(dict_test['pos|pos']+dict_test['neg|pos'])
        f1=(2.0*precision*recall)/(precision+recall)
        
    else:
        import numpy as np
        from nltk.probability import FreqDist
        from nltk.classify import SklearnClassifier
        from sklearn.feature_extraction.text import TfidfTransformer
        from sklearn.feature_selection import SelectKBest, chi2
        from sklearn.naive_bayes import MultinomialNB
        from sklearn.pipeline import Pipeline
        pipeline = Pipeline([('tfidf', TfidfTransformer()),
                     ('chi2', SelectKBest(chi2, k='all')),
                     ('nb', MultinomialNB())])
        classif = SklearnClassifier(pipeline)

        pos=[FreqDist(prep_text(data[ID])) for ID in IDs if int(getrating[ID][0])>3]
        neg=[FreqDist(prep_text(data[ID])) for ID in IDs if int(getrating[ID][0])<3]

        add_label = lambda lst, lab: [(x, lab) for x in lst]


        classif.train(add_label(pos[:lpr], 'pos') + add_label(neg[:lnr], 'neg'))

        l_pos = np.array(classif.classify_many(pos[lpr:]))
        l_neg = np.array(classif.classify_many(neg[lnr:]))
        pospos, posneg,negpos,negneg = (l_pos == 'pos').sum(), (l_pos == 'neg').sum(),(l_neg == 'pos').sum(), (l_neg == 'neg').sum()

        accuracy=float(pospos+negneg)/(pospos+ posneg+negpos+negneg)
        recall=float(pospos)/(pospos+posneg)
        precision=float(pospos)/(pospos+negpos)
        f1=(2.0*precision*recall)/(precision+recall)
    
    overview = pd.DataFrame([[lid,posrevs,negrevs,count_tok,count_utok,accuracy,recall,precision,f1,str(round(split_ratio*100,2))+'%',lpr+lnr,lpr,lnr,ufeats]], columns=['reviews','reviews+','reviews-','token','#unique','accuracy','recall','precision','f1-measure','learn_%','learn_rev','learn_rev+','learn_rev-','#u_feats'])

    
    return(overview)

In [2]:
import re
import sqlite3
import pandas as pd
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['raw','clean','spellcheck','ner','stopwords','tfidf']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.1, SEED=42, db=db))
result = pd.concat(frames)
result.index=dfs
result1=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
raw,91506,45748,45758,12300580,525762,0.837097,0.886071,0.806699,0.844524,10.0%,9150,4626,4524,121354
clean,91504,45747,45757,12291834,262256,0.833050,0.891579,0.798606,0.842536,10.0%,9150,4491,4659,75424
spellcheck,91504,45747,45757,12480304,184233,0.829881,0.897469,0.791052,0.840908,10.0%,9150,4491,4659,63325
ner,91504,45747,45757,12335770,131083,0.824720,0.901057,0.782166,0.837413,10.0%,9150,4491,4659,42861
stopwords,90818,45340,45478,1662519,3849,0.847633,0.898873,0.814942,0.854852,10.0%,9081,4540,4541,3843
tfidf,90818,45340,45478,1662519,3849,0.853922,0.867525,0.844145,0.855675,10.0%,9081,4540,4541,3843


In [3]:
import re
import sqlite3
import pandas as pd
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['raw','clean','spellcheck','ner','stopwords','tfidf']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
result = pd.concat(frames)
result.index=dfs
result2=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
raw,91506,45748,45758,12300580,525762,0.857649,0.918284,0.818926,0.865764,50.0%,45753,22876,22877,344605
clean,91504,45747,45757,12291834,262256,0.851263,0.926415,0.806065,0.862060,50.0%,45752,22794,22958,184880
spellcheck,91504,45747,45757,12480304,184233,0.845843,0.926371,0.798580,0.857742,50.0%,45752,22794,22958,137281
ner,91504,45747,45757,12335770,131083,0.834980,0.925282,0.784472,0.849078,50.0%,45752,22794,22958,95663
stopwords,90818,45340,45478,1662519,3849,0.853950,0.903633,0.820959,0.860314,50.0%,45409,22739,22670,3849
tfidf,90818,45340,45478,1662519,3849,0.857715,0.873236,0.845877,0.859339,50.0%,45409,22739,22670,3849


In [4]:
import re
import sqlite3
import pandas as pd
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['raw','clean','spellcheck','ner','stopwords','tfidf']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.8, SEED=42, db=db))
result = pd.concat(frames)
result.index=dfs
result3=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
raw,91506,45748,45758,12300580,525762,0.868758,0.918877,0.833716,0.874228,80.0%,73204,36663,36541,459741
clean,91504,45747,45757,12291834,262256,0.860609,0.918410,0.822802,0.867981,80.0%,73203,36616,36587,234838
spellcheck,91504,45747,45757,12480304,184233,0.857658,0.919834,0.817660,0.865742,80.0%,73203,36616,36587,167798
ner,91504,45747,45757,12335770,131083,0.845965,0.915124,0.803462,0.855665,80.0%,73203,36616,36587,118493
stopwords,90818,45340,45478,1662519,3849,0.852621,0.903721,0.816011,0.857629,80.0%,72654,36418,36236,3849
tfidf,90818,45340,45478,1662519,3849,0.856584,0.873347,0.840833,0.856782,80.0%,72654,36418,36236,3849


In [5]:
import sqlite3
import pandas as pd
import re
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['stopwords','skip_ner']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
result = pd.concat(frames)
result.index=dfs
result4=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
stopwords,90818,45340,45478,1662519,3849,0.853950,0.903633,0.820959,0.860314,50.0%,45409,22739,22670,3849
skip_ner,88915,44337,44578,910351,3791,0.817693,0.876689,0.782187,0.826746,50.0%,44457,22279,22178,3767


In [6]:
pd.concat([result1.iloc[-1].to_frame().T,result2.iloc[-1].to_frame().T,result3.iloc[-1].to_frame().T])

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
tfidf,90818,45340,45478,1662519,3849,0.853922,0.867525,0.844145,0.855675,10.0%,9081,4540,4541,3843
tfidf,90818,45340,45478,1662519,3849,0.857715,0.873236,0.845877,0.859339,50.0%,45409,22739,22670,3849
tfidf,90818,45340,45478,1662519,3849,0.856584,0.873347,0.840833,0.856782,80.0%,72654,36418,36236,3849


In [7]:
pd.concat([result2[:3],result4.iloc[-1].to_frame().T])

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
raw,91506,45748,45758,12300580,525762,0.857649,0.918284,0.818926,0.865764,50.0%,45753,22876,22877,344605
clean,91504,45747,45757,12291834,262256,0.851263,0.926415,0.806065,0.86206,50.0%,45752,22794,22958,184880
spellcheck,91504,45747,45757,12480304,184233,0.845843,0.926371,0.79858,0.857742,50.0%,45752,22794,22958,137281
skip_ner,88915,44337,44578,910351,3791,0.817693,0.876689,0.782187,0.826746,50.0%,44457,22279,22178,3767


# Reliabilität

In [3]:
import sqlite3
import pandas as pd
import re
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['tfidf','tfidf','tfidf','tfidf','tfidf']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED='None', db=db))
result = pd.concat(frames)
result.index=dfs
result5=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
tfidf,90818,45340,45478,1662519,3849,0.860336,0.871554,0.853202,0.862280,50.0%,45409,22560,22849,3849
tfidf,90818,45340,45478,1662519,3849,0.859169,0.876499,0.846970,0.861481,50.0%,45409,22652,22757,3849
tfidf,90818,45340,45478,1662519,3849,0.861525,0.878133,0.848254,0.862935,50.0%,45409,22799,22610,3849
tfidf,90818,45340,45478,1662519,3849,0.860336,0.878252,0.846856,0.862268,50.0%,45409,22736,22673,3849
tfidf,90818,45340,45478,1662519,3849,0.856901,0.871490,0.846559,0.858843,50.0%,45409,22657,22752,3849


In [4]:
max(result5['accuracy'])-min(result5['accuracy'])

0.00462463388315093

# SVM

In [13]:
def word_feats(words):
    return dict([(word, True) for word in words.split()])

def prep_text(String):
    return re.sub('[\s]+',' ',String).replace('.','').strip().split(' ')

def classify2(data,split_ratio,SEED,db):
    
    import sqlite3
    import re
    import pandas as pd
    from nltk.classify import NaiveBayesClassifier
    import random

    amazon_sql = 'data/amazon/amazon.db' #db -> id+rating

    # Lade Amazon_raw
    conn=sqlite3.connect(amazon_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID, text,rating FROM dvd')
    getrating={ID:rating for ID, text,rating in cur}
    conn.close() 

    count_tok=0
    count_utok=set()

    delete=[]
    for key,value in data.items():
        try:
            text=re.sub('[\s]+',' ',data[key].replace('.','')).strip().lower()
            temp=text.split(' ')
            count_utok.update(set(temp))
            count_tok+=len(temp)

            data[key]=text
        except:
            delete.append(key)
            
    for key in delete:
        data.pop(key, None)

    count_utok=len(count_utok)
    count_tok,count_utok
    
    posrevs=len(['' for ID in data.keys() if int(getrating[ID][0])>3])
    negrevs=len(['' for ID in data.keys()  if int(getrating[ID][0])<3])
    

    IDs=list(data.keys())
    if type(SEED) is int:
        random.seed(SEED)
    random.shuffle(IDs)

    l=int(len(IDs)*split_ratio)
    trainIDs=IDs[:l]
    testIDs=IDs[l:]
    lid=len(IDs)

    posids=[data[ID] for ID in trainIDs if int(getrating[ID][0])>3]
    negids=[data[ID] for ID in trainIDs if int(getrating[ID][0])<3]
    
    lpr=len(posids)
    lnr=len(negids)
    
    
    ufeats=set()
    for text in posids+negids:
        try:
            text=re.sub('[\s]+',' ',text.replace('.','')).strip().lower()
            temp=text.split(' ')
            ufeats.update(set(temp))
        except:
            pass
    ufeats=len(ufeats)
    
    if db!='tfidf':
        pos_feats = [(word_feats(f), 'positive') for f in posids ]
        neg_feats = [(word_feats(f), 'negative') for f in negids ]
        trainfeats = pos_feats + neg_feats

        classifier = NaiveBayesClassifier.train(trainfeats)

        dict_test={'pos|pos':0,'pos|neg':0,'neg|pos':0,'neg|neg':0,}


        for ID in testIDs:
            rating=int(getrating[ID][0])
            if rating<3:
                rating='neg'
            elif rating>3:
                rating='pos'
            true_rating=rating

            rating=[(label, classifier.prob_classify(word_feats(data[ID])).prob(label)) for label in ['positive','negative']][0][1]
            if rating<=0.5:
                rating='neg'
            elif rating>=0.5:
                rating='pos'
            pred_rating=rating
            dict_test[true_rating+'|'+pred_rating]+=1

            test_matrix = pd.DataFrame(index=['pos','neg'], columns=['pos','neg'])
            test_matrix = test_matrix.fillna('')

            for tag,n in dict_test.items():
                col, row = tag.split('|')
                test_matrix.at[col, row] = n

        accuracy=float(dict_test['pos|pos']+dict_test['neg|neg'])/(sum(dict_test.values()))
        recall=float(dict_test['pos|pos'])/(dict_test['pos|pos']+dict_test['pos|neg'])
        precision=float(dict_test['pos|pos'])/(dict_test['pos|pos']+dict_test['neg|pos'])
        f1=(2.0*precision*recall)/(precision+recall)
        
    else:
        import numpy as np
        from nltk.probability import FreqDist
        from nltk.classify import SklearnClassifier
        from sklearn.feature_extraction.text import TfidfTransformer
        from sklearn.feature_selection import SelectKBest, chi2
        from sklearn.naive_bayes import MultinomialNB
        from sklearn.pipeline import Pipeline
        from sklearn.linear_model import SGDClassifier
        pipeline = Pipeline([('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                   alpha=1e-3, random_state=42,
                                   max_iter=5, tol=None))])
        classif = SklearnClassifier(pipeline)

        pos=[FreqDist(prep_text(data[ID])) for ID in IDs if int(getrating[ID][0])>3]
        neg=[FreqDist(prep_text(data[ID])) for ID in IDs if int(getrating[ID][0])<3]

        add_label = lambda lst, lab: [(x, lab) for x in lst]


        classif.train(add_label(pos[:lpr], 'pos') + add_label(neg[:lnr], 'neg'))

        l_pos = np.array(classif.classify_many(pos[lpr:]))
        l_neg = np.array(classif.classify_many(neg[lnr:]))
        pospos, posneg,negpos,negneg = (l_pos == 'pos').sum(), (l_pos == 'neg').sum(),(l_neg == 'pos').sum(), (l_neg == 'neg').sum()

        accuracy=float(pospos+negneg)/(pospos+ posneg+negpos+negneg)
        recall=float(pospos)/(pospos+posneg)
        precision=float(pospos)/(pospos+negpos)
        f1=(2.0*precision*recall)/(precision+recall)
    
    overview = pd.DataFrame([[lid,posrevs,negrevs,count_tok,count_utok,accuracy,recall,precision,f1,str(round(split_ratio*100,2))+'%',lpr+lnr,lpr,lnr,ufeats]], columns=['reviews','reviews+','reviews-','token','#unique','accuracy','recall','precision','f1-measure','learn_%','learn_rev','learn_rev+','learn_rev-','#u_feats'])

    
    return(overview)

import sqlite3
import pandas as pd
import re
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]

dfs=['tfidf']
for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
    
    
for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify2(data=data, split_ratio=0.5, SEED=42, db=db))
result = pd.concat(frames)
result.index=['NB','SVM']
result6=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
NB,90818,45340,45478,1662519,3849,0.857715,0.873236,0.845877,0.859339,50.0%,45409,22739,22670,3849
SVM,90818,45340,45478,1662519,3849,0.845647,0.886023,0.818751,0.851059,50.0%,45409,22739,22670,3849


# Konsistenz

In [27]:
import re
import sqlite3
import pandas as pd
text_sql = 'data/text_processing/text_processing.db' #db -> texts
frames=[]


def drop_word(String,text):
    text=[w for w in text.split(' ') if w!=String]
    return ' '.join(text)

dfs=['tfidf']

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:text.lower() for ID, text in cur if text!=None and len(text.replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
    
for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:drop_word('kelly',text.lower()) for ID, text in cur if text!=None and len(drop_word('kelly',text.lower()).replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))

for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:drop_word('unschlagbar',text.lower()) for ID, text in cur if text!=None and len(drop_word('unschlagbar',text.lower()).replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
    
for db in dfs:

    conn=sqlite3.connect(text_sql)
    cur = conn.cursor()
    cur.execute('SELECT ID,text FROM '+db)
    data={ID:drop_word('staffel',text.lower()) for ID, text in cur if text!=None and len(drop_word('staffel',text.lower()).replace('.','').strip())>0}
    conn.close()

    frames.append(classify(data=data, split_ratio=0.5, SEED=42, db=db))
    
result = pd.concat(frames)
result.index=['tfidf','kelly','unschlagbar','staffel']
result1=result.copy()
result

,reviews,reviews+,reviews-,token,#unique,accuracy,recall,precision,f1-measure,learn_%,learn_rev,learn_rev+,learn_rev-,#u_feats
tfidf,90818,45340,45478,1662519,3849,0.857715,0.873236,0.845877,0.859339,50.0%,45409,22739,22670,3849
kelly,90818,45340,45478,1662314,3848,0.857826,0.873368,0.845969,0.859451,50.0%,45409,22739,22670,3848
unschlagbar,90817,45339,45478,1662298,3848,0.857671,0.872155,0.845925,0.858840,50.0%,45408,22796,22612,3848
staffel,90798,45322,45476,1650225,3848,0.860217,0.872430,0.851244,0.861707,50.0%,45399,22660,22739,3848
